In [38]:
# import required libraries
import os
import datetime
from azure.ai.ml.constants import AssetTypes

from azureml.core import (
    Workspace,
    Datastore,
    Environment,
    RunConfiguration,
    ScriptRunConfig,
    Experiment,
    Workspace,
    Run,
    Model
)
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice

from azure.identity import DefaultAzureCredential


In [39]:
# required definitons
environment_name = 'my-env'
model_name = 'nnunet-devops-test-2'
#Define the size and location of our deployed compute
cpu_cores = 2
memory_gb = 8
location = 'eastus'
service_name = 'my-service'
model_download_dir = './model_outputs'

# required information
environment_image = 'nnunet.azurecr.io/prediction_env'
experiment_name = 'nnunet-devops-test-2'
entry_script = 'score.py'
source_directory = 'src'
run_id = "d134788d-094b-4cfa-b38b-c2eced07ba53" #"d6c83f1e-97a7-48a1-b4db-a308de7bd9e2" # gets the latest if empty

In [41]:
workspace = Workspace.from_config()

# get experiment
experiment = Experiment(workspace, experiment_name)
# get experiment's latest run id
if run_id:
    latest_run = Run(experiment, run_id) 
else:
    latest_run = next(experiment.get_runs())
    
model_path = f"dataset/{run_id}/nnUNet/"

ds = workspace.get_default_datastore()
ds.download(target_path="./model_outputs", prefix=model_path, overwrite=True)

# register the model

# model registration procedure
registered_model = Model.register(
                model_path=os.path.join(model_download_dir, model_path),
                model_name=model_name,
                workspace=workspace)

# Model files downloading process overrides model_download_directory.
# Thus, no need to remove directory.


Downloaded dataset/d134788d-094b-4cfa-b38b-c2eced07ba53/nnUNet/nnUNet/2d/Task005_Prostate/nnUNetTrainer__nnUNetPlansv2.1/all/debug.json, 1 files out of an estimated total of 106
Downloaded dataset/d134788d-094b-4cfa-b38b-c2eced07ba53/nnUNet/nnUNet/2d/Task005_Prostate/nnUNetTrainer__nnUNetPlansv2.1/all/model_final_checkpoint.model.pkl, 2 files out of an estimated total of 106
Downloaded dataset/d134788d-094b-4cfa-b38b-c2eced07ba53/nnUNet/nnUNet/2d/Task005_Prostate/nnUNetTrainer__nnUNetPlansv2.1/all/postprocessing.json, 3 files out of an estimated total of 106
Downloaded dataset/d134788d-094b-4cfa-b38b-c2eced07ba53/nnUNet/nnUNet/2d/Task005_Prostate/nnUNetTrainer__nnUNetPlansv2.1/all/progress.png, 4 files out of an estimated total of 106
Downloaded dataset/d134788d-094b-4cfa-b38b-c2eced07ba53/nnUNet/nnUNet/2d/Task005_Prostate/nnUNetTrainer__nnUNetPlansv2.1/all/validation_raw/prostate_00.nii.gz, 5 files out of an estimated total of 106
Downloaded dataset/d134788d-094b-4cfa-b38b-c2eced07ba5

In [42]:
# you don't have to create environment if you createfd before with same docker
env = Environment.from_docker_image(name=environment_name, image=environment_image)
env.inferencing_stack_version='latest'


In [43]:
#Define configs for the deployment
inference_config = InferenceConfig(
    entry_script=entry_script, environment=env, source_directory=source_directory
)
aci_config = AciWebservice.deploy_configuration(cpu_cores = cpu_cores, memory_gb = memory_gb,location=location)

#Deloy the service
service = Model.deploy(
        workspace=workspace,
        name=service_name,
        models=[registered_model],
        inference_config=inference_config,
        deployment_config=aci_config,
        overwrite=True
    )
service.wait_for_deployment(show_output=True)

/tmp/ipykernel_9856/3269267396.py:8: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration. 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(


service.wait_for_deployment(show_output=True)


In [47]:
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2023-07-27 08:27:59+00:00 Creating Container Registry if not exists.
2023-07-27 08:27:59+00:00 Registering the environment.
2023-07-27 08:28:00+00:00 Building image.